In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import *
import seaborn as sns

In [ ]:
df=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/train.csv',na_values=['?','-999','Error','xxxxxxxx'])

In [ ]:
df=df.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)

In [ ]:
for i in df.columns:
  if df[i].dtype=='float64':
    df[i]=df[i].fillna(df[i].mean())
  else:
    df[i]=df[i].fillna(method='ffill')

In [ ]:
df.fillna('Both',inplace=True)
df['joining_date']=df['joining_date'].apply(lambda x:int(x.split('-')[0]))
df.rename(columns={'joining_date':'joining_year'},inplace=True)
df1=pd.get_dummies(df,drop_first=True)

In [ ]:
x=df1.drop('churn_risk_score',axis=1)
y=df1['churn_risk_score']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,stratify=y,random_state=42)

In [ ]:
import xgboost
xgc=xgboost.XGBClassifier()
xgc.fit(x_train,y_train)

In [ ]:
xgcpred=xgc.predict(x_test)
print(f1_score(y_test,xgcpred,average='macro'))

In [ ]:
xgc=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=3, missing=None, n_estimators=119, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1,seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgc.fit(x_train,y_train)
xgcpred=xgc.predict(x_test)
print(f1_score(y_test,xgcpred,average='macro'))

In [ ]:
test=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',na_values=['?','-999','Error','xxxxxxxx'])
test.head()

In [ ]:
test=test.drop(['customer_id','Name','security_no','referral_id','last_visit_time'],axis=1)
test.shape

In [ ]:
for i in test.columns:
  if test[i].dtype=='float64':
    test[i]=test[i].fillna(test[i].mean())
  else:
    test[i]=test[i].fillna(method='ffill')

In [ ]:
test.fillna('Both',inplace=True)
test['joining_date']=test['joining_date'].apply(lambda x:int(x.split('-')[0]))
test.rename(columns={'joining_date':'joining_year'},inplace=True)
test1=pd.get_dummies(test,drop_first=True)


In [ ]:
rdpred=xgc.predict(test1)


In [ ]:
df2=pd.read_csv('../input/churn-risk-rate-hackerearth-ml/test.csv',usecols=['customer_id'])
df2['churn_risk_score']=rdpred
df2

In [ ]:
df2.to_csv('output.csv',index=False)